In [ ]:
pip install selenium

In [1]:
#Importing necessary libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import time
import warnings
import csv
warnings.filterwarnings('ignore')

In [2]:
#Specifying the path and launching chromedriver
PATH = r'chromedriver.exe'
driver = webdriver.Chrome()

In [ ]:
a_links = [] #List to store the links of the restaurants
page=1
x=1
while page<40:   #Number of pages during scraping was around 40 so thats why we used this as a limit
    try: #Try and except in order to get the links of a page
        name=driver.find_element_by_xpath(f"(//*[contains(@class, 'bHGqj Cj b')])[{x}]") #Use appropriate xpath
        a_links.append(name.get_attribute("href"))
        #increment x by 1
        x+=1
    except Exception as e:
        break    
    if(x%31==0): #Click the button for the next set of restaurants
        if page == 1:
            python_button =driver.find_element_by_xpath('//*[@id="EATERY_LIST_CONTENTS"]/div[2]/div/a[1]')
            python_button.click()
            time.sleep(2) #Wait 2sec to load
            page+=1
        else:
            python_button =driver.find_element_by_xpath('//*[@id="EATERY_LIST_CONTENTS"]/div[2]/div/a[2]')
            python_button.click()
            time.sleep(2)
            page+=1 
        x=1
pd.DataFrame(a_links).to_csv("Restaurant_Links.csv") #Save to csv all the links

In [3]:
#Read the previous csv and drop duplicates. Duplicates are a result of sponsored restaurants
a_links = pd.read_csv('LINKS_FILE_CSV',header=None) #Here we open the links file
a_links = a_links[1]
a_links.drop_duplicates(inplace = True)
a_links.reset_index(drop = True, inplace = True)
a_links

0      https://www.tripadvisor.com/Restaurant_Review-...
1      https://www.tripadvisor.com/Restaurant_Review-...
2      https://www.tripadvisor.com/Restaurant_Review-...
3      https://www.tripadvisor.com/Restaurant_Review-...
4      https://www.tripadvisor.com/Restaurant_Review-...
                             ...                        
259    https://www.tripadvisor.com/Restaurant_Review-...
260    https://www.tripadvisor.com/Restaurant_Review-...
261    https://www.tripadvisor.com/Restaurant_Review-...
262    https://www.tripadvisor.com/Restaurant_Review-...
263    https://www.tripadvisor.com/Restaurant_Review-...
Name: 1, Length: 264, dtype: object

In [4]:
import csv
import time
from selenium import webdriver


# Prepare CSV file
csvFile = open("reviews4.csv", "w", newline='', encoding="utf-8") #Open the file where the reviews are going to be saved
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['Business_Name','Score','Username','Rating_date','Date_of_Visit','Title','Review','Review_Distribution','Age_Gender_Location'])
data = [] #Various lists that are going to help for saving the different data that we want to acquire
review_distribution = []
user_inf = [] 
my_xpath = '//div[@class="info_text pointer_cursor"]'
x_path_x = '//*[@id="BODY_BLOCK_JQUERY_REFLOW"]/span/div[4]'

for i in range(len(a_links)):
    URL = a_links[i] #Start going through the links
    driver.get(URL)
    business_name = driver.title.split(',')[0] #Split the link in order to add a variable that will change the pages later
    x=1
    try: #Use appropriate xpaths
        number_of_reviews=driver.find_element_by_xpath('//*[@id="taplc_detail_filters_rr_resp_0"]/div/div[1]/div/div[2]/div[4]/div/div[2]/div[1]/div[2]/label/span[2]').text
        number_of_reviews = re.search(r"\d+", number_of_reviews)
        num=int(number_of_reviews.group())
        split_link=URL.split('Reviews') 
        while x<=num: #Same as previously we scrape the pages of the reviews of each restaurant
            if x==1:
                driver.get(split_link[0]+'Reviews-or'+split_link[1]) #Here we add the variable that was mentioned 
                x=15
            else:
                try:
                    
                    driver.get(split_link[0]+'Reviews-or'+f"{x}-"+split_link[1])#Here we add the variable that was mentioned 
                    x=x+15
                except Exception as e:
                    break
            # Find and click the More link (to load all reviews)
            try:
                driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
                time.sleep(4) # Wait for reviews to load
            except:
                pass
            elements = driver.find_elements_by_css_selector(".info_text.pointer_cursor")
            elements2 = driver.find_elements_by_css_selector(".prw_rup.prw_reviews_stay_date_hsx")
            reviews = driver.find_elements_by_xpath("//div[@class='ui_column is-9']")
            num_page_items = min(len(reviews), 15)
            users_info = driver.find_elements_by_xpath(my_xpath)
            # Loop through the reviews found
            for i, user in enumerate(users_info):
                # get the score, date, title and review, date of visit, etc
                score_class = reviews[i].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
                score = score_class.split("_")[3]
                rating_date = reviews[i].find_element_by_xpath(".//span[@class='ratingDate']").get_attribute("title")
                date_of_visit = elements2[i].text
                title = reviews[i].find_element_by_xpath(".//span[@class='noQuotes']").text
                review = reviews[i].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", "")
                username = elements[i].find_element_by_tag_name("div").text

                user.click() #here it is clicking the user in order for the pop up window to show
                inf = driver.find_elements_by_xpath('//*[@class="countsReviewEnhancementsItem"]')
                time.sleep(1) #Use time sleeps to load the data 
                data.append(inf)
                try:
                    member_info = driver.find_element_by_xpath('//*[@class="memberdescriptionReviewEnhancements"]')
                    time.sleep(1)
                    b = member_info.text
                    
                except:
                    b = None
                    pass
                try:
                    review_dist = driver.find_element_by_xpath('//*[@class="wrap container histogramReviewEnhancements"]')
                    time.sleep(1)
                    a = review_dist.text
                   
                except:
                    a = None
                    pass
                time.sleep(1)
                users_info = driver.find_element_by_xpath(x_path_x).click() #Saving data
                'Business_Name','Score','Username','Rating_date','Date_of_Visit','Title','Review','Review_Distribution','Age_Gender_Location'
                csvWriter.writerow(( business_name, score, username, rating_date, date_of_visit, title, review,a,b))# Close CSV file and browser
                try:
                    users_info[i+1].click() #Moving on
                except:
                    pass
    except:
        pass
csvFile.close() #Close the csv file
  